In [2]:
%pip install -qU \
    replicate \
    langchain \
    sentence_transformers \
    pdf2image \
    pdfminer \
    pdfminer.six \
    unstructured


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install ctransformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [6]:
import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

In [7]:
from langchain.llms import CTransformers

llama_model = CTransformers(
    model = "C:\\Users\\adity\\Downloads\\llama-2-7b-chat.ggmlv3.q2_K.bin",
    model_type = "llama",
    config = {'max_new_tokens':1000,
              'temperature':0.75,
              'context_length':2000}
)

In [8]:
#load the external data source
from langchain.document_loaders import OnlinePDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader('C:\\Users\\adity\\Desktop\\Chat UI\\Data')
documents = loader.load()

#Get text splits from document
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

#Use the embedding model
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2" # embedding model
model_kwargs = {"device": "cpu"}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

#Use vector store to store embeddings
vectorstore = FAISS.from_documents(all_splits, embeddings)

In [9]:
def md(t):
  display(Markdown(t))

In [10]:
# Query 1
from langchain.chains import ConversationalRetrievalChain
chain = ConversationalRetrievalChain.from_llm(llama_model, vectorstore.as_retriever(), return_source_documents=True)

chat_history = []
query = "what are the symptoms of Frostbite?"
result = chain({"question": query, "chat_history": chat_history})
md(result['answer'])

c:\Users\adity\Desktop\Chat UI\env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


 The symptoms of frostbite are numbness, discoloration and thickening. 

In [12]:
# Query 2
query = "symptoms of cough?"
result1 = chain({"question": query, "chat_history": chat_history})
md(result1['answer'])


The main symptoms of cough caused by chronic bronchitis are:
• A persistent, deep, and painful cough that brings up a greenish yellow phlegm or sputum.
• Wheezing after coughing is common.
• Coughing may bring up mucusal material that is sometimes bloody.

In [13]:
# Query 2
query = "symptoms of cough?"
result2 = chain({"question": query, "chat_history": chat_history})
md(result2['answer'])

 The symptoms of cough in chronic bronchitis include the production of thick, yellow-green phlegm or sputum, wheezing (or a tight feeling in the chest), and difficulty breathing.

In [14]:
#General Query
query = input()
result = chain({"question": query, "chat_history": chat_history})
md(result['answer'])

 The answer to this question can be found in the text provided. According to the text, nosebleeds (or "epistaxis") are caused by inflammation of the nose, or rhinitis, which is a common symptom of adult respiratory distress syndrome (ARDS).

In [15]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# Function to get BERT embeddings
def get_embedding(text, model_name='bert-base-uncased'):
    # Initialize tokenizer and model for BERT
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    # Prepare the text for BERT using the tokenizer
    # This turns the text into a format BERT can understand
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)

    # Disable gradient calculation for performance
    with torch.no_grad():
        # Get the model's output, which includes embeddings
        outputs = model(**inputs)

    # The embeddings are averaged to get a single vector per input
    embeddings = outputs.last_hidden_state.mean(1)
    return embeddings.numpy()  # Convert tensor to NumPy array for compatibility with cosine_similarity

In [16]:
def compare_responses(response1, response2, model_name='bert-base-uncased'):
    # Convert responses to embeddings
    emb1 = get_embedding(response1, model_name)
    emb2 = get_embedding(response2, model_name)

    # Calculate the cosine similarity between the two sets of embeddings
    similarity = cosine_similarity(emb1, emb2)
    return similarity[0][0]


# Compare the two responses and print the similarity score
similarity_score = compare_responses(result1['answer'], result2['answer'])
print("Cosine similarity score:", similarity_score)

Cosine similarity score: 0.95428795
